# 思路：先对两张图像分别分类后，再用分类后得到的分类图像做差，如果结果图像中的值为0，说明没有发生变化，若不为0，则说明发生了变化。
# 问题：数据集哪里找？训练需要数据集，然后再找两幅时间不相同的图  
该论文提供了数据集：https://www.mdpi.com/2072-4292/16/13/2355

In [ ]:
# CNN需要原图 + 已分类的图，CNN自动提取特征，不需要自己提取特征
# 监督式学习分类：你需提供特征, label
# 非监督式学习分类：你需提供特征